In [1]:
# import libraries
import keras
import math
import pandas as pd
import numpy as np
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
import time
from torch.utils.data import Dataset, DataLoader
import pickle
pd.set_option('display.max_rows', 500)
import os
import tensorflow as tf
import torch
import torch.nn as nn
from math import sqrt
# import rmse from sklearn
from sklearn.metrics import mean_squared_error


# define random seeds for Neural Networks
torch.manual_seed(0)
np.random.seed(0)
tf.random.set_seed(0)
# ignore warnings jupyter notebook
import warnings
warnings.filterwarnings('ignore')

# ------ FUNCTIONS ------

In [3]:
# function to merge data for hague 
def merge_trejectory_data(results, trajectory, direction):
    data = pd.DataFrame()
    for intersection_name in results[trajectory][direction]['raw']:
        intersection = results[trajectory][direction]['raw'][intersection_name]
        intersection = intersection.rename(columns={"cars": intersection_name})
        intersection = intersection.set_index(pd.DatetimeIndex(intersection['timestamp']))
        intersection = intersection.drop(columns=['timestamp'])
        data = pd.merge(data, intersection, left_index=True, right_index=True, how='outer')
    data.dropna(inplace=True)
    return data

In [4]:
# function to merge all intersection data to one dataframe for metr and pems
def merge_trejectory_data_metr(results):
    data = pd.DataFrame()
    for intersection_name in results['raw']:
        intersection = results['raw'][intersection_name]
        intersection = intersection.rename(columns={"cars": intersection_name})
        intersection = intersection.set_index(pd.DatetimeIndex(intersection['timestamp']))
        intersection = intersection.drop(columns=['timestamp'])
        data = pd.merge(data, intersection, left_index=True, right_index=True, how='outer')
    data.dropna(inplace=True)
    return data

In [5]:
# function to preprocess the data for LSTM
def preprocess_df(df,n_obs, n_features, sequence_length):
    #do scaling:
    # scaler = StandardScaler()
    df_train = df.values
    train_X, train_y = df_train[:, :n_obs], df_train[:, -n_features]
    scl= 'None'
    # scl = scaler.fit(train_X) # fit only on training data
    # train_X = scl.transform(train_X)
    train_X = train_X.reshape((train_X.shape[0], sequence_length, n_features))
    return train_X, train_y, scl

In [6]:
# function to convert data to time series 
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [7]:
# define LSTM model
class LSTM_uni(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, device = 'mps',layer_dim=1, dropout_prob = 0.2):
        super(LSTM_uni, self).__init__()
        self.hidden_dim = hidden_dim # number of hidden units in hidden state
        self.layer_dim = layer_dim # number of stacked lstm layers
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_dim, output_dim) # fully connected layer

    def forward(self, x, future=False):
        # input x is expected to be of shape (batch_dim, seq_dim, feature_dim)
        # hidden and cell states are expected along with input x in LSTMs = (h_0, c_0)
        # Initialize hidden state with zeros (layer_dim, batch_size, hidden_dim)
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=device).requires_grad_()
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=device).requires_grad_()

        # LSTM output is Outputs: output, (h_n, c_n)
        # output is of shape (batch_dim, seq_dim, hidden_dim), h_n and c_n are of shape (layer_dim, batch_dim, hidden_dim)
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = out[:, -1, :] # only take the last output of the sequence
        out = self.fc(out) # fully connected layer

        return out

In [13]:
# standard training model
def train_model(model, train_X,train_y, loss_fn, optimiser, device, epochs=100):
    history = {}
    best_loss = np.inf
    tolerance = 0
    max_tolerance = 5
    history['train_loss'] = []

    train_X_loader = DataLoader(train_X, batch_size=128, shuffle=False)
    train_y_loader = DataLoader(train_y, batch_size=128, shuffle=False)

    for epoch in range(epochs):
        history[epoch] = []
        running_loss = 0.0
        for bx, data in enumerate(zip(train_X_loader,train_y_loader)):
            X = data[0].to(device)
            y = data[1].to(device)
            bt = model(X)
            loss = loss_fn(bt.reshape(-1), y.reshape(-1)) # calculate loss for input and recreated output
            history[epoch].append(loss.item())
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
            running_loss += loss.item()
        epoch_loss = running_loss/train_X.shape[0]

        # Check if validation model has improved
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            tolerance = 0
        else:
            tolerance += 1
            if tolerance >= max_tolerance:
                print("loss hasn't improved for {} epochs. Early stopping!".format(max_tolerance))
                break
        history['train_loss'].append(epoch_loss)
    
    return history, model

In [9]:
# function to evaluate the model
def model_evaluation( model, test_X, device):
    test_X_loader = DataLoader(test_X, batch_size=128, shuffle=False)
    model = model.eval()
    preds = []
    with torch.no_grad():
        for bx, data in enumerate(test_X_loader):
            X = data.to(device)
            bt = model(X)
            preds.append(bt.cpu().numpy())
    preds = np.vstack(preds)
    preds = preds.reshape(-1)
    return preds

In [10]:
# function to load checkpoint of model
def load_ckp(checkpoint_fpath, model, optimizer):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model, optimizer

In [14]:
# function for first test then train model (test_then_train_protocol) for continious data stream
def test_then_train_weighted_model(base_dict_col, dict_col, data, ae_score, number_of_cols, target, results_load_path, results_save_path, hidden_size, output_pred, num_layers, dropout, device,sequence_length, epoch_train):

    result_dict = {}

    # ------------------------------------ test data processing ---------------------------------------- #
    df = data.copy(deep=True)
    df = df[dict_col.keys()].mul(dict_col.values(), axis=1)
    isct_inc = data.shape[1]
    df = df.astype('float32')
    n_features = isct_inc # number of features (correlated intersections)
    n_obs = sequence_length * n_features # number of columns in the input
    reframed = series_to_supervised(df, sequence_length, output_pred)
    train_X, train_y, scl = preprocess_df(reframed, n_obs, n_features, sequence_length)



    # # # ------------------------------------ modelling ---------------------------------------------- #
    # define model, loss function and optimizer
    model = LSTM_uni(input_dim = n_features, hidden_dim = hidden_size, output_dim = output_pred, layer_dim = num_layers, dropout_prob= dropout, device = device)
    model = model.to(device)
    loss_fn = torch.nn.MSELoss()
    optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
    model, optimiser = load_ckp(results_load_path, model, optimiser)


    # ------------------------------------ testing ---------------------------------------------- #
    start = time.time()
    yhat = model_evaluation(model, train_X , device)
    end = time.time()
    result_dict['RMSE'] = sqrt(mean_squared_error(yhat,train_y))
    result_dict['MAE'] = mean_absolute_error(yhat,train_y)
    result_dict['R2'] = r2_score(yhat,train_y)
    result_dict['df'] = pd.DataFrame({"Real":train_y,"Predicted":yhat})
    result_dict['eval_time'] = end - start
    result_dict['data_shape'] = train_X.shape
    # print("RMSE: {}".format(result_dict['RMSE']))



    # ------------------------------------ train data processing ---------------------------------------- #
    dict_col = base_dict_col.copy()
    top_corr_df = ae_score.corr()[target].sort_values(ascending=False)
    top_corr_df[number_of_cols:] = 0 # get the top correlated intersections
    top_corr_df = top_corr_df.fillna(1) # fill na with 1 to avoid nan values in the data
    for key,val in dict_col.items():
        dict_col[key] = top_corr_df[key]
    df = data.copy(deep=True)
    df = df[dict_col.keys()].mul(dict_col.values(), axis=1)
    isct_inc = data.shape[1]
    df = df.astype('float32')
    n_features = isct_inc # number of features (correlated intersections)
    n_obs = sequence_length * n_features # number of columns in the input
    reframed = series_to_supervised(df, sequence_length, output_pred)
    train_X, train_y, scl = preprocess_df(reframed, n_obs, n_features, sequence_length)


   # ------------------------------------ training ---------------------------------------------- #
    start = time.time()
    history, model = train_model(model, train_X,train_y, loss_fn, optimiser, device, epochs=epoch_train)
    end = time.time()
    checkpoint = {'state_dict': model.state_dict(),'optimizer': optimiser.state_dict()}
    torch.save(checkpoint, results_save_path)
    # print("Training time: {}".format(end-start))
    result_dict['shape'] = data.shape
    result_dict['history'] = history
    result_dict['intersedctions'] = top_corr_df
    result_dict['data'] = data
    result_dict['train_time'] = end-start
    
    return result_dict, dict_col

In [15]:
# fuction to train the base model (LSTM) using first 50% of the data
def train_base_model(base_dict_col, data, ae_score, number_of_cols, target, results_load_path, results_save_path, hidden_size, output_pred, num_layers, dropout, device, sequence_length, epoch_train):
    result_dict = {}
    dict_col = base_dict_col.copy()
        
    # ------------------------------------ data processing ---------------------------------------- #
    top_corr_df = ae_score.corr()[target].sort_values(ascending=False)
    top_corr_df[number_of_cols:] = 0 # get the top correlated intersections
    top_corr_df = top_corr_df.fillna(1) # fill na with 1 to avoid nan values in the data
    for key,val in dict_col.items():
        dict_col[key] = top_corr_df[key]
    df = data.copy(deep=True)
    df = df[dict_col.keys()].mul(dict_col.values(), axis=1)
    isct_inc = data.shape[1]
    # # print("correlated intersections: {}".format(isct_inc))
    # df = data[isct_inc].copy(deep=True)
    # df = df.mul(top_corr_df, axis=1)
    df = df.astype('float32')
    n_features = isct_inc # number of features (correlated intersections)
    n_obs = sequence_length * n_features # number of columns in the input
    reframed = series_to_supervised(df, sequence_length, output_pred)
    train_X, train_y, scl = preprocess_df(reframed, n_obs, n_features, sequence_length)


    # # # ------------------------------------ modelling ---------------------------------------------- #
    # define model, loss function and optimizer
    model = LSTM_uni(input_dim = n_features, hidden_dim = hidden_size, output_dim = output_pred, layer_dim = num_layers, dropout_prob= dropout, device = device)
    model = model.to(device)
    loss_fn = torch.nn.MSELoss()
    optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
    
    # if not os.path.exists(results_load_path):
    #     model, optimiser = load_ckp(results_load_path, model, optimiser)
        # print("Model loaded")
    # ------------------------------------ training ---------------------------------------------- #
    print("Training base model")
    start = time.time()
    history, model = train_model(model, train_X,train_y, loss_fn, optimiser, device, epochs=epoch_train)
    end = time.time()
    checkpoint = {'state_dict': model.state_dict(),'optimizer': optimiser.state_dict()}
    torch.save(checkpoint, results_save_path)
    # print("Training time: {}".format(end-start))
    result_dict['history'] = history
    result_dict['intersections'] = top_corr_df
    result_dict['data'] = data
    result_dict['train_time'] = end-start
        
    return result_dict,dict_col


In [16]:
# function for base model evaluation
def evaluate_base_model(dict_col, data, ae_score, number_of_cols, target, results_load_path, results_save_path, hidden_size, output_pred, num_layers, dropout, device, sequence_length, epoch_train):

    result_dict = {}
    
    # ------------------------------------ data processing ---------------------------------------- #
    df = data.copy(deep=True)
    df = df[dict_col.keys()].mul(dict_col.values(), axis=1)
    isct_inc = data.shape[1]
    # # print("correlated intersections: {}".format(isct_inc))
    # df = data[isct_inc].copy(deep=True)
    # df = df.mul(top_corr_df, axis=1)
    df = df.astype('float32')
    n_features = isct_inc # number of features (correlated intersections)
    n_obs = sequence_length * n_features # number of columns in the input
    reframed = series_to_supervised(df, sequence_length, output_pred)
    train_X, train_y, scl = preprocess_df(reframed, n_obs, n_features, sequence_length)


    # # # ------------------------------------ modelling ---------------------------------------------- #
    # define model, loss function and optimizer
    model = LSTM_uni(input_dim = n_features, hidden_dim = hidden_size, output_dim = output_pred, layer_dim = num_layers, dropout_prob= dropout, device = device)
    model = model.to(device)
    loss_fn = torch.nn.MSELoss()
    optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
    # check if there is a pre-trained model
    
    if os.path.exists(results_load_path):
        model, optimiser = load_ckp(results_load_path, model, optimiser)


    # ------------------------------------ testing ---------------------------------------------- #
    start = time.time()
    yhat = model_evaluation(model, train_X , device)
    end = time.time()
    result_dict['RMSE'] = sqrt(mean_squared_error(yhat,train_y))
    result_dict['MAE'] = mean_absolute_error(yhat,train_y)
    result_dict['R2'] = r2_score(yhat,train_y)
    result_dict['df'] = pd.DataFrame({"Real":train_y,"Predicted":yhat})
    result_dict['eval_time'] = end - start
    result_dict['data_shape'] = train_X.shape
    print("RMSE: of base model is - {}".format(result_dict['RMSE']))

    return result_dict

# ------- MAIN ----------

## ---------------------------- hague data processing ---------------------------- 

In [29]:
# define path and load results
data_name = 'hague'
base_result_path = f'../results/{data_name}/real_time_modeling'
with open(f'../data/{data_name}/processed/featured_fpds_raw.pickle', 'rb') as f:
    results = pickle.load(f)

outlier_model_name_list = ["PW-AE"]
time_window_list = [1, 3, 6, 12, 24, 24*7, 24*30]
outlier_update = False # change to True if you want to update the outlier model

In [25]:
# declare model specific variables
target_intersections={"T1":{"North":"K504", "South":"K561"}, # get target intersections for each trajectory and direction
                      "T2":{"North":"K703", "South":"K206"}}
threshold = 1
epoch_train = 75
epoch_retrain = 15
batch_size = 64
learning_rate = 0.1
hidden_size = 32
num_layers = 1
dropout = 0.2
sequence_length = 12
output_pred = 1 # number of time steps to predict
base_train_portion = 0.5 # percentage of data to use for training
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

In [ ]:
# main script for real-time training and testing of LSTM_OWAM model
for outlier_model_name in outlier_model_name_list:
    errors={}
    print(f"=================================== \n\n Processing {outlier_model_name}")

    # load data of correlated results from pickle file
    with open(f'../results/{data_name}/outlier_scores/{outlier_model_name}/correlated_results.pickle', 'rb') as f:
        correlated_results = pickle.load(f)

    # start loop over trajectories
    for trajectory in results.keys():
        errors[trajectory]={}
        for direction in results[trajectory]:
            print(f"\n \n ------------------------ Starting trajectory {trajectory}, direction {direction} and threshold: {threshold} ------------------------")
            key = trajectory+"_"+direction
            target = target_intersections[trajectory][direction]
            errors[trajectory][direction]={}
            errors[trajectory][direction][threshold]={}

            # ------------------------------------ data processing ---------------------------------------- #
            ae_score = correlated_results[key] # AE scores of the current trajectory and direction
            data = merge_trejectory_data(results, trajectory, direction)# get raw data of the current trajectory and direction
            data = data[[target] + [col for col in data.columns if col != target]] # move target intersection to the first column
            base_dict_col = {} # fixed dictionary to keep the columns at same positions
            for col in data.columns:
                base_dict_col[col] = 0
            data_size = data.shape[0]
            model_start_time  = data.index[0]
            model_end_time = data.index[-1]
            baseline_time = data.iloc[int(data_size*base_train_portion)].name
            base_ae = ae_score[ae_score.index<=baseline_time]
            stream_ae = ae_score[ae_score.index>baseline_time]
            base_data = data[data.index<=baseline_time]
            stream_data = data[data.index>(baseline_time - pd.DateOffset(hours=1))]
            number_of_cols = math.ceil(len(ae_score.columns)*threshold) # number of outlier weighted intersections
            if number_of_cols==0: # if threshold is 0, then use the target intersection only
                number_of_cols=1

            # ------------------------------------ training base model ---------------------------------------- #
            # print(f"training from {model_start_time} to {baseline_time}")
            exp_name = f'univariate_{outlier_model_name}_{key}_real_time_base_model.pt'
            results_save_path = os.path.join(base_result_path, exp_name)
            base_model_path = results_save_path
            results_load_path = results_save_path
            results_load_weighted_path = "None"
            
            result_dict,dict_col = train_base_model(base_dict_col, base_data, base_ae, number_of_cols, target, results_load_path, base_model_path, hidden_size, output_pred, num_layers, dropout, device, sequence_length, epoch_train)
            no_update_model_results = evaluate_base_model(dict_col, stream_data, stream_ae, number_of_cols, target, results_load_path, results_save_path, hidden_size, output_pred, num_layers, dropout, device, sequence_length, epoch_train)
            errors[trajectory][direction][threshold]['No_update'] = no_update_model_results
            errors[trajectory][direction][threshold]['base_model'] = result_dict

            # # # ------------------------------------ incremental training ---------------------------------------- # 

            for time_window in time_window_list:
                if time_window==1:
                    epoch_retrain = 3
                else:
                    epoch_retrain = 15

                print(f"\n\n Processing time window {time_window}\n\n ===================================")
                incremental_updating_time = baseline_time + pd.DateOffset(hours=time_window)
                baseline_updating_time = baseline_time 
                time_key = str(time_window)
                errors[trajectory][direction][threshold][time_key] = {}
                errors[trajectory][direction][threshold][time_key]['incremental_weighted_update'] = {}


                while(incremental_updating_time<model_end_time):
                    # print(f"\n\ntraining from {baseline_time} to {incremental_updating_time}")
                    incremental_ae = stream_ae[(stream_ae.index>baseline_updating_time) & (stream_ae.index<=incremental_updating_time)]
                    incremental_data = stream_data[(stream_data.index>(baseline_updating_time - pd.DateOffset(hours=1))) & (stream_data.index<=incremental_updating_time)]
                    exp_name_weighted = f'univariate_real_time_{outlier_model_name}_incremental_weighted_{incremental_updating_time}.pkl'
                    results_load_path = results_save_path
                    results_save_path = os.path.join(base_result_path, exp_name)

                    # ------------------------------------ training incremental models ---------------------------------------- #
                    # print("training incremental weighted update model")
                    if outlier_update:
                        model_results,dict_col = test_then_train_weighted_model(base_dict_col, dict_col, incremental_data, incremental_ae, number_of_cols, target, results_load_path, results_save_path, hidden_size, output_pred, num_layers, dropout, device,sequence_length, epoch_retrain)
                    else:
                        model_results,dict_col = test_then_train_weighted_model(base_dict_col, dict_col, incremental_data, base_ae, number_of_cols, target, results_load_path, results_save_path, hidden_size, output_pred, num_layers, dropout, device,sequence_length, epoch_retrain)
                    errors[trajectory][direction][threshold][time_key]['incremental_weighted_update'][incremental_updating_time] = model_results

                    baseline_updating_time = incremental_updating_time # update baseline time
                    incremental_updating_time = incremental_updating_time + pd.DateOffset(hours=time_window) # update incremental time
                

    # ------------------------------------ save results ---------------------------------------- #
    with open(os.path.join(base_result_path, f'univariate_real_time_{outlier_model_name}_threshold_{threshold}_no_out_update.pkl'), 'wb') as f:
        pickle.dump(errors, f)

## ---------------------------- METR-LA data processing ---------------------------- 

In [30]:
# define experiment variables
data_name = 'METR-LA'
base_result_path = f'../results/{data_name}/real_time_modeling'
with open(f'../data/{data_name}/processed/featured_fpds_raw.pickle', 'rb') as f:
    results = pickle.load(f)

outlier_model_name_list = ["PW-AE"]
time_window_list = [1, 3, 6, 12, 24, 24*7, 24*30]
outlier_update = False # change to True if you want to update the outlier model

In [34]:
# define model specific parametrers
target_intersections = list(np.random.choice(list(results['raw'].keys()), 5, replace=False)) # select 5 intersections randomly from the intersections
# always check if the generated target_intersections are same (sometimes np random does not work)
threshold = 0.1
epoch_train = 75
epoch_retrain = 15
batch_size = 64
learning_rate = 0.1
hidden_size = 32
num_layers = 1
dropout = 0.2
sequence_length = 12
output_pred = 1 # number of time steps to predict
base_train_portion = 0.5 # percentage of data to use for training
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

In [ ]:
# main script to train and test LSTM_OWAM model in real time
for outlier_model_name in outlier_model_name_list:
    errors={}
    print(f"=================================== \n\n Processing {outlier_model_name}")

    # load data of correlated results from pickle file
    with open(f'../results/{data_name}/outlier_scores/{outlier_model_name}/correlated_results.pickle', 'rb') as f:
        correlated_results = pickle.load(f)

    # start loop over trajectories
    for target in target_intersections:
        print(f"\n \n ------------------------ Starting target: {target} and threshold: {threshold} ------------------------")
        key = target
        errors[target]={}
        errors[target][threshold]={}


        # ------------------------------------ data processing ---------------------------------------- #
        ae_score = correlated_results['df'] # AE scores of the current trajectory and direction
        data = merge_trejectory_data_metr(results)# get raw data of the current trajectory and direction
        data = data[[target] + [col for col in data.columns if col != target]] # move target intersection to the first column
        base_dict_col = {} # fixed dictionary to keep the columns at same positions
        for col in data.columns:
            base_dict_col[col] = 0
        data_size = data.shape[0]
        model_start_time  = data.index[0]
        model_end_time = data.index[-1]
        baseline_time = data.iloc[int(data_size*base_train_portion)].name
        base_ae = ae_score[ae_score.index<=baseline_time]
        stream_ae = ae_score[ae_score.index>baseline_time]
        base_data = data[data.index<=baseline_time]
        stream_data = data[data.index>(baseline_time - pd.DateOffset(hours=1))]
        number_of_cols = math.ceil(len(ae_score.columns)*threshold) # number of outlier weighted intersections
        if number_of_cols==0: # if threshold is 0, then use the target intersection only
            number_of_cols=1

        # ------------------------------------ training base model ---------------------------------------- #
        # print(f"training from {model_start_time} to {baseline_time}")
        exp_name = f'univariate_{outlier_model_name}_{key}_real_time_base_model.pt'
        results_save_path = os.path.join(base_result_path, exp_name)
        base_model_path = results_save_path
        results_load_path = results_save_path
        results_load_weighted_path = "None"
        
        result_dict,dict_col = train_base_model(base_dict_col, base_data, base_ae, number_of_cols, target, results_load_path, base_model_path, hidden_size, output_pred, num_layers, dropout, device, sequence_length, epoch_train)
        no_update_model_results = evaluate_base_model(dict_col, stream_data, stream_ae, number_of_cols, target, results_load_path, results_save_path, hidden_size, output_pred, num_layers, dropout, device, sequence_length, epoch_train)
        errors[target][threshold]['No_update'] = no_update_model_results
        errors[target][threshold]['base_model'] = result_dict

        # # # ------------------------------------ incremental training ---------------------------------------- # 

        for time_window in time_window_list:
            if time_window==1:
                epoch_retrain = 3
            else:
                epoch_retrain = 15
            print(f"\n\n Processing time window {time_window}\n\n ===================================")
            incremental_updating_time = baseline_time + pd.DateOffset(hours=time_window)
            baseline_updating_time = baseline_time 
            time_key = str(time_window)
            errors[target][threshold][time_key] = {}
            errors[target][threshold][time_key]['incremental_weighted_update'] = {}


            while(incremental_updating_time<model_end_time):
                # print(f"\n\ntraining from {baseline_time} to {incremental_updating_time}")
                incremental_ae = stream_ae[(stream_ae.index>baseline_updating_time) & (stream_ae.index<=incremental_updating_time)]
                incremental_data = stream_data[(stream_data.index>(baseline_updating_time - pd.DateOffset(hours=1))) & (stream_data.index<=incremental_updating_time)]
                exp_name_weighted = f'univariate_real_time_{outlier_model_name}_incremental_weighted_{incremental_updating_time}.pkl'
                results_load_path = results_save_path
                results_save_path = os.path.join(base_result_path, exp_name)

                # ------------------------------------ training incremental models ---------------------------------------- #
                # print("training incremental weighted update model")
                if outlier_update:
                    model_results,dict_col = test_then_train_weighted_model(base_dict_col, dict_col, incremental_data, incremental_ae, number_of_cols, target, results_load_path, results_save_path, hidden_size, output_pred, num_layers, dropout, device,sequence_length, epoch_retrain)
                else:
                    model_results,dict_col = test_then_train_weighted_model(base_dict_col, dict_col, incremental_data, base_ae, number_of_cols, target, results_load_path, results_save_path, hidden_size, output_pred, num_layers, dropout, device,sequence_length, epoch_retrain)
                
                errors[target][threshold][time_key]['incremental_weighted_update'][incremental_updating_time] = model_results

                baseline_updating_time = incremental_updating_time # update baseline time
                incremental_updating_time = incremental_updating_time + pd.DateOffset(hours=time_window) # update incremental time
            

    # ------------------------------------ save results ---------------------------------------- #
    with open(os.path.join(base_result_path, f'univariate_real_time_{outlier_model_name}_threshold_{threshold}_no_out_update.pkl'), 'wb') as f:
        pickle.dump(errors, f)


## ----------- PEMS-BAY data processing -------

In [33]:
data_name = 'PEMS-BAY'
base_result_path = f'../results/{data_name}/real_time_modeling'
with open(f'../data/{data_name}/processed/featured_fpds_raw.pickle', 'rb') as f:
    results = pickle.load(f)

outlier_model_name_list = ["PW-AE"]
time_window_list = [1, 3, 6, 12, 24, 24*7, 24*30]
outlier_update = False # change to True if you want to update the outlier model

In [57]:
# define the parameters for the LSTM model
target_intersections = list(np.random.choice(list(results['raw'].keys()), 5, replace=False)) # select 5 intersections randomly the intersections
# declare variables
threshold = 0.5
epoch_train = 150
epoch_retrain = 35
batch_size = 64
learning_rate = 0.01
hidden_size = 32
num_layers = 1
dropout = 0.2
sequence_length = 12
output_pred = 1 # number of time steps to predict
base_train_portion = 0.5 # percentage of data to use for training
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

In [ ]:
# main function to run the LSTM model for real-time outlier detection
for outlier_model_name in outlier_model_name_list:
    errors={}
    print(f"=================================== \n\n Processing {outlier_model_name}")

    # load data of correlated results from pickle file
    with open(f'../results/{data_name}/outlier_scores/{outlier_model_name}/correlated_results.pickle', 'rb') as f:
        correlated_results = pickle.load(f)

    # start loop over trajectories
    for target in target_intersections:
        print(f"\n \n ------------------------ Starting target: {target} and threshold: {threshold} ------------------------")
        key = target
        errors[target]={}
        errors[target][threshold]={}


        # ------------------------------------ data processing ---------------------------------------- #
        ae_score = correlated_results['df'] # AE scores of the current trajectory and direction
        data = merge_trejectory_data_metr(results)# get raw data of the current trajectory and direction
        data = data[[target] + [col for col in data.columns if col != target]] # move target intersection to the first column
        base_dict_col = {} # fixed dictionary to keep the columns at same positions
        for col in data.columns:
            base_dict_col[col] = 0
        data_size = data.shape[0]
        model_start_time  = data.index[0]
        model_end_time = data.index[-1]
        baseline_time = data.iloc[int(data_size*base_train_portion)].name
        base_ae = ae_score[ae_score.index<=baseline_time]
        # change column names to integers for base_ae
        base_ae.columns = [int(col) for col in base_ae.columns]
        stream_ae = ae_score[ae_score.index>baseline_time]
        # change column names to integers for stream_ae
        stream_ae.columns = [int(col) for col in stream_ae.columns]
        base_data = data[data.index<=baseline_time]
        stream_data = data[data.index>(baseline_time - pd.DateOffset(hours=1))]
        number_of_cols = math.ceil(len(ae_score.columns)*threshold) # number of outlier weighted intersections
        if number_of_cols==0: # if threshold is 0, then use the target intersection only
            number_of_cols=1

        # ------------------------------------ training base model ---------------------------------------- #
        # print(f"training from {model_start_time} to {baseline_time}")
        exp_name = f'univariate_{outlier_model_name}_{key}_real_time_base_model.pt'
        results_save_path = os.path.join(base_result_path, exp_name)
        base_model_path = results_save_path
        results_load_path = results_save_path
        results_load_weighted_path = "None"
        
        result_dict,dict_col = train_base_model(base_dict_col, base_data, base_ae, number_of_cols, target, results_load_path, base_model_path, hidden_size, output_pred, num_layers, dropout, device, sequence_length, epoch_train)
        no_update_model_results = evaluate_base_model(dict_col, stream_data, stream_ae, number_of_cols, target, results_load_path, results_save_path, hidden_size, output_pred, num_layers, dropout, device, sequence_length, epoch_train)
        errors[target][threshold]['No_update'] = no_update_model_results
        errors[target][threshold]['base_model'] = result_dict

        # # # ------------------------------------ incremental training ---------------------------------------- # 

        for time_window in time_window_list:
            print(f"\n\n Processing time window {time_window}\n\n ===================================")
            incremental_updating_time = baseline_time + pd.DateOffset(hours=time_window)
            baseline_updating_time = baseline_time 
            time_key = str(time_window)
            errors[target][threshold][time_key] = {}
            errors[target][threshold][time_key]['incremental_weighted_update'] = {}


            while(incremental_updating_time<model_end_time):
                # print(f"\n\ntraining from {baseline_time} to {incremental_updating_time}")
                incremental_ae = stream_ae[(stream_ae.index>baseline_updating_time) & (stream_ae.index<=incremental_updating_time)]
                incremental_data = stream_data[(stream_data.index>(baseline_updating_time - pd.DateOffset(hours=1))) & (stream_data.index<=incremental_updating_time)]
                exp_name_weighted = f'univariate_real_time_{outlier_model_name}_incremental_weighted_{incremental_updating_time}.pkl'
                results_load_path = results_save_path
                results_save_path = os.path.join(base_result_path, exp_name)

                # ------------------------------------ training incremental models ---------------------------------------- #
                # print("training incremental weighted update model")
                if outlier_update:
                    model_results,dict_col = test_then_train_weighted_model(base_dict_col, dict_col, incremental_data, incremental_ae, number_of_cols, target, results_load_path, results_save_path, hidden_size, output_pred, num_layers, dropout, device,sequence_length, epoch_retrain)
                else:
                    model_results,dict_col = test_then_train_weighted_model(base_dict_col, dict_col, incremental_data, base_ae, number_of_cols, target, results_load_path, results_save_path, hidden_size, output_pred, num_layers, dropout, device,sequence_length, epoch_retrain)
                errors[target][threshold][time_key]['incremental_weighted_update'][incremental_updating_time] = model_results

                baseline_updating_time = incremental_updating_time # update baseline time
                incremental_updating_time = incremental_updating_time + pd.DateOffset(hours=time_window) # update incremental time
            

    # ------------------------------------ save results ---------------------------------------- #
    with open(os.path.join(base_result_path, f'univariate_real_time_{outlier_model_name}_threshold_{threshold}_no_out_update.pkl'), 'wb') as f:
        pickle.dump(errors, f)

# ------ END ------